In [2]:
# %%
from ecallisto_ng.data_download.downloader import get_ecallisto_data
from ecallisto_ng.burst_list.utils import load_burst_list
from datetime import timedelta, datetime
from PIL import Image
import random
import os
from tqdm import tqdm
import pandas as pd


In [3]:
FOLDER = "/mnt/nas05/data01/vincenzo/ecallisto/data"
RESOLUTION = (256, 256)
SAVE_IMG = False  # Save DF instead of image with full resolution.
SAVE_TYPE = "parquet"
burst_type = '3'
resample_delta = timedelta(minutes=15) / RESOLUTION[0]  # Ist nicht perfekt, aber geht
instrument = "GLASGOW_01"
start_datetime = datetime(2023, 2, 14, 8, 52, 0)

In [4]:
start_datetime

datetime.datetime(2023, 2, 14, 8, 52)

In [5]:
def save_image(df, path):
    """
    Save an image to a file.

    Parameters:
    df (pandas.DataFrame): The dataframe containing the image data.
    path (str): The path to save the image to.
    """
    # Get the image data from the dataframe
    image_data = df.values

    # Convert the image data to a PIL Image
    image = Image.fromarray(image_data)

    # Save the image to the specified path
    image.save(path)


In [6]:
end_datetime = start_datetime + timedelta(minutes=15)

dfs = get_ecallisto_data(
    start_datetime,
    end_datetime,
    instrument_name=instrument,
    download_from_local=True,
)


In [7]:
dfs['GLASGOW_01']

,45.000000,45.125000,45.250000,45.438000,45.688000,45.750000,46.000000,46.125000,46.313000,46.500000,...,79.125000,79.313004,79.625000,79.688004,79.875000,80.188004,80.250000,80.500000,80.688004,80.813004
2023-02-14 08:52:00.228,172,175,169,163,163,166,167,168,168,167,...,172,173,172,172,171,171,174,174,173,170
2023-02-14 08:52:00.478,172,174,170,163,163,166,167,168,168,167,...,172,173,172,173,173,173,175,175,173,171
2023-02-14 08:52:00.728,172,174,169,163,162,165,167,168,168,168,...,173,174,172,173,173,172,175,175,174,172
2023-02-14 08:52:00.978,172,174,169,163,163,167,167,169,167,168,...,173,173,172,173,173,173,173,174,174,171
2023-02-14 08:52:01.228,172,173,169,163,163,166,166,168,169,167,...,172,172,173,173,172,172,175,174,173,171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-14 09:06:58.845,172,175,170,161,162,164,165,165,166,165,...,171,171,171,172,172,171,174,174,172,171
2023-02-14 09:06:59.095,172,175,170,162,163,166,166,168,166,165,...,171,172,172,172,172,172,173,173,173,170
2023-02-14 09:06:59.345,171,175,169,161,161,164,165,166,166,166,...,172,173,171,172,171,171,172,172,172,171
2023-02-14 09:06:59.595,172,175,169,162,163,165,166,167,166,165,...,172,172,172,173,173,175,175,176,174,173


In [9]:
for _, df in dfs.items():
    if instrument != df.attrs["FULLNAME"]:
        continue
    if SAVE_IMG:
        # Resample
        df = df.resample(resample_delta).max()
        assert (
            df.shape[0] > 200
        ), f"Number of rows should be more than 200, got {df.shape[0]}"
        # Maybe keep only good frequencies?
        # Background sub?
        ## Path to save the image to
        # It's FOLDER / instrument / burst type / start_datetime.png
        path = os.path.join(
            FOLDER,
            instrument,
            burst_type,
            start_datetime.strftime("%Y-%m-%d_%H-%M-%S") + ".png",
        )
        os.makedirs(os.path.dirname(path), exist_ok=True)
        save_image(df.T, path)
    else:
        assert (df.index.max() - df.index.min()) > pd.Timedelta(
            10, unit="minutes"
        )
        # Maybe keep only good frequencies?
        # Background sub?
        ## Path to save the image to
        # It's FOLDER / instrument / burst type / start_datetime.png
        path = os.path.join(
            FOLDER,
            instrument,
            burst_type,
            start_datetime.strftime("%Y-%m-%d_%H-%M-%S") + ".parquet",
        )
        os.makedirs(os.path.dirname(path), exist_ok=True)
        print(path)
        df.to_parquet(path)


/mnt/nas05/data01/vincenzo/ecallisto/data/GLASGOW_01/3/2023-02-14_08-52-00.parquet


In [8]:
pd.read_parquet('/mnt/nas05/data01/vincenzo/ecallisto/data/GLASGOW_01/0/2023-02-14_08-52-00.parquet')

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/nas05/data01/vincenzo/ecallisto/data/GLASGOW_01/0/2023-02-14_08-52-00.parquet'